In [9]:
from multinomial import MultinomialLogisticRegression
from preprocess import Preprocess
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
raw_train_data = "../data/train.csv"
raw_test_data = "../data/test.csv"

In [14]:
preproc = Preprocess()
raw_data = preproc.load_data(raw_train_data)

# cols = [
#     "lat",
#     "Z1000",
#     "Z200",
#     "TMQ",
#     "PSL",
#     "U850",
#     "VBOT",
#     "TS",
#     "QREFHT",
#     "UBOT",
#     "time",
# ]

# raw_data = raw_data[cols + ["Label"]]

train_df, train_data = preproc.preprocess_data(raw_data, drop_cols=["SNo", "time"])
np.random.shuffle(train_data)
X_train, y_train, X_valid, y_valid = preproc.train_valid_split(
    train_data, test_size=0.33, random_state=42
)

print(train_df.columns)

X_train = preproc.normalize_data(X_train)
X_valid = preproc.normalize_data(X_valid)

Index(['lat', 'lon', 'TMQ', 'U850', 'V850', 'UBOT', 'VBOT', 'QREFHT', 'PS',
       'PSL', 'T200', 'T500', 'PRECT', 'TS', 'TREFHT', 'Z1000', 'Z200', 'ZBOT',
       'month', 'Label'],
      dtype='object')


In [15]:
# Create and train the model.
model = MultinomialLogisticRegression(learning_rate=0.1, num_iterations=1000)
model.fit(X_train, y_train, collist=raw_data.columns)

# Make predictions.
predictions = model.predict(X_valid)
print(train_df.columns)

model.get_metrics(y_valid, predictions, return_values=False)

Epoch 0, cross entropy loss: 1.0986122886681096


Epoch 100, cross entropy loss: 0.8295995308663886
Epoch 200, cross entropy loss: 0.7770580626953214
Epoch 300, cross entropy loss: 0.755675514793755
Epoch 400, cross entropy loss: 0.7424951326476658
Epoch 500, cross entropy loss: 0.7329129311226104
Epoch 600, cross entropy loss: 0.7254817087634824
Epoch 700, cross entropy loss: 0.7194970993182681
Epoch 800, cross entropy loss: 0.7145299752898114
Epoch 900, cross entropy loss: 0.7102980429963403
Index(['lat', 'lon', 'TMQ', 'U850', 'V850', 'UBOT', 'VBOT', 'QREFHT', 'PS',
       'PSL', 'T200', 'T500', 'PRECT', 'TS', 'TREFHT', 'Z1000', 'Z200', 'ZBOT',
       'month', 'Label'],
      dtype='object')
Confusion Matrix:
[[9977  491 1099]
 [ 305  298    1]
 [1063   59 1478]] 

Accuracy:
0.7957 

Precision:
0.6014 

Recall:
0.6415 

F1 Score:
0.6208


In [16]:
# test
preproc = Preprocess()
raw_data = preproc.load_data(raw_test_data)
# raw_data = raw_data[cols + ["SNo"]]

test_df, test_data = preproc.preprocess_data(
    raw_data, drop_cols=["SNo", "time"], is_test=True
)

print(test_df.columns)

test_data = preproc.normalize_data(test_data)
y_pred_test = model.predict(test_data)

submition = raw_data["SNo"].reset_index().copy()
submition["Label"] = pd.Series(y_pred_test)
submition.drop("index", axis=1, inplace=True)
submition.to_csv(f"predictions_{datetime.now()}.csv", index=False)


Index(['lat', 'lon', 'TMQ', 'U850', 'V850', 'UBOT', 'VBOT', 'QREFHT', 'PS',
       'PSL', 'T200', 'T500', 'PRECT', 'TS', 'TREFHT', 'Z1000', 'Z200', 'ZBOT',
       'month'],
      dtype='object')


In [7]:
model.train_errors

AttributeError: 'MultinomialLogisticRegression' object has no attribute 'train_errors'